<a href="https://colab.research.google.com/github/Farhanahoque251/Drug-Discovery-with-Python/blob/main/(Cancer-EGFR)Lipi_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install chembl_webresource_client rdkit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
jsonschema 4.19.2 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.
referencing 0.32.0 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.


In [ ]:
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import Lipinski
from rdkit.Chem import Descriptors
# Search for compounds related to coronavirus in ChEMBL
target_query = new_client.target
target = target_query.filter(target_synonym__icontains='EGFR').only('target_chembl_id')[4]

In [ ]:


# Get bioactivities for the target
activity = new_client.activity
res = activity.filter(target_chembl_id=target['target_chembl_id'], pchembl_value__isnull=False).filter(standard_type="IC50")

# Collect unique compound IDs
compounds = {x['molecule_chembl_id'] for x in res}

# Fetch molecule details and apply Lipinski's Rule of Five
molecules = []
for compound_id in compounds:
    mol_data = new_client.molecule.filter(molecule_chembl_id=compound_id).only('molecule_structures')
    smiles = mol_data[0]['molecule_structures']['canonical_smiles']
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        mw = Descriptors.MolWt(mol)  # Updated line
        hbd = Lipinski.NumHDonors(mol)
        hba = Lipinski.NumHAcceptors(mol)
        clogp = Chem.Crippen.MolLogP(mol)
        if mw <= 500 and hbd <= 5 and hba <= 10 and clogp <= 5:
            molecules.append({'SMILES': smiles, 'MolecularWeight': mw, 'HBD': hbd, 'HBA': hba, 'CLogP': clogp})

# Print or process the filtered molecules
for molecule in molecules:
    print(molecule)

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

# Assuming 'molecules' contains the filtered molecules from earlier processing
# Let's create RDKit Mol objects from SMILES
mols = [Chem.MolFromSmiles(mol['SMILES']) for mol in molecules]

# Generate images of the molecules
images_2D = [Draw.MolToImage(mol) for mol in mols]

# Show the images
for img in images_2D:
    display(img)

In [ ]:
# Count of molecules meeting criteria
num_molecules = len(molecules)
print(f"Number of molecules meeting Lipinski's Rule of Five criteria: {num_molecules}")

# To get the number of rows and columns
num_rows = num_molecules
num_columns = len(molecules[0])  # Assuming all molecules have the same columns
print(f"Number of rows: {num_rows}, Number of columns: {num_columns}")

# To display the molecules data
for molecule in molecules:
    print(molecule)

Number of molecules meeting Lipinski's Rule of Five criteria: 0


IndexError: ignored

In [ ]:
import pandas as pd

# Assuming 'molecules' contains the filtered data
data = {
    'SMILES': [mol['SMILES'] for mol in molecules],
    'MolecularWeight': [mol['MolecularWeight'] for mol in molecules],
    'HBD': [mol['HBD'] for mol in molecules],
    'HBA': [mol['HBA'] for mol in molecules],
    'CLogP': [mol['CLogP'] for mol in molecules]
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

                                               SMILES  MolecularWeight  HBD  \
0                 Brc1cccc(Nc2ncnc3cc4[nH]cnc4cc23)c1          340.184    2   
1   Cn1cc(-c2cc(C(=O)/N=c3\[nH]c4ccccc4n3CC(C)(C)O...          390.447    2   
2                      Nc1ccc2ncnc(Nc3cccc(Br)c3)c2c1          315.174    2   
3   C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(O...          499.619    2   
4            O=C(NCc1ccccc1)c1cc(NCc2cc(O)ccc2O)ccc1O          364.401    5   
5        O=C(NCCc1ccc(F)cc1)c1cc(NCc2cc(O)ccc2O)ccc1O          396.418    5   
6             COc1cc(OC)c2c(=O)c(-c3cccc(Cl)c3)coc2c1          316.740    0   
7           O=C(NCCc1ccccn1)c1cc(NCc2cc(O)ccc2O)ccc1O          379.416    5   
8            CN(CO)/N=N/c1ccc2ncnc(Nc3cccc(Br)c3)c2c1          387.241    2   
9         CC(=O)N(C)/N=N/c1ccc2ncnc(Nc3cccc(C)c3)c2c1          334.383    1   
10  CC(C)(O)Cn1/c(=N/C(=O)c2ccnc(-c3cccnc3)c2)[nH]...          387.443    2   
11  Cn1ncc2c1OCCCCCN1/C(=N/C(=O)c3cc(F)nc-2c3)Nc2c..

In [ ]:
!pip install py3Dmol

In [ ]:
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import Lipinski
from rdkit.Chem import AllChem

# Search for compounds related to cancer in ChEMBL
target_query = new_client.target
target = target_query.filter(target_synonym__icontains='cancer').only('target_chembl_id')[0]

# Get bioactivities for the target
activity = new_client.activity
res = activity.filter(target_chembl_id=target['target_chembl_id'], pchembl_value__isnull=False).filter(standard_type="IC50")

# Collect unique compound IDs
compounds = {x['molecule_chembl_id'] for x in res}

# Fetch molecule details and apply Lipinski's Rule of Five
molecules = []
for compound_id in compounds:
    try:
        mol_data = new_client.molecule.filter(molecule_chembl_id=compound_id).only('molecule_structures')
        smiles = mol_data[0]['molecule_structures']['canonical_smiles']
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            mw = Chem.Descriptors.MolWt(mol)
            hbd = Lipinski.NumHDonors(mol)
            hba = Lipinski.NumHAcceptors(mol)
            clogp = Chem.Crippen.MolLogP(mol)
            if mw <= 500 and hbd <= 5 and hba <= 10 and clogp <= 5:
                molecules.append({'SMILES': smiles, 'MolecularWeight': mw, 'HBD': hbd, 'HBA': hba, 'CLogP': clogp})

                # Fetch bioactivity data for the compound
                compound_activities = activity.filter(molecule_chembl_id=compound_id)

                # Display bioactivity data for the compound
                if compound_activities:
                    print(f"Compound {compound_id} has bioactivities:")
                    for act in compound_activities:
                        print(f"  - Activity Type: {act['standard_type']}, Value: {act['standard_value']} {act['standard_units']}")
                else:
                    print(f"No bioactivity data found for Compound {compound_id}")

                # Generate 3D conformers and visualize
                mol_3d = Chem.AddHs(mol)
                AllChem.EmbedMolecule(mol_3d, AllChem.ETKDG())
                AllChem.UFFOptimizeMolecule(mol_3d)
                pdb_block = Chem.MolToPDBBlock(mol_3d)
                # Visualize using Py3Dmol (replace this with your visualization code)
                # viewer = py3Dmol.view(width=300, height=300)
                # viewer.addModel(pdb_block, 'pdb')
                # viewer.setStyle({'stick': {}})
                # viewer.zoomTo()
                # viewer.show()

    except Exception as e:
        print(f"Error processing Compound {compound_id}: {str(e)}")